In [1]:
from importlib.metadata import version

print("파이토치 버전:", version("torch"))

파이토치 버전: 2.8.0+cu126


# 연습문제 4.1: 피드 포워드 신경망과 어텐션 모듈에 있는 파라미터 수

In [2]:
!wget https://bit.ly/3SZbhlj -O gpt.py

--2025-11-09 09:43:04--  https://bit.ly/3SZbhlj
Resolving bit.ly (bit.ly)... 67.199.248.10, 67.199.248.11
Connecting to bit.ly (bit.ly)|67.199.248.10|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://raw.githubusercontent.com/rickiepark/llm-from-scratch/refs/heads/main/ch04/01_main-chapter-code/gpt.py [following]
--2025-11-09 09:43:04--  https://raw.githubusercontent.com/rickiepark/llm-from-scratch/refs/heads/main/ch04/01_main-chapter-code/gpt.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9675 (9.4K) [text/plain]
Saving to: ‘gpt.py’

gpt.py              100%[===================>]   9.45K  --.-KB/s    in 0s      

2025-11-09 09:43:04 (33.3 MB/s) - ‘gpt.py’ saved [9675/9675]



In [3]:
from gpt import TransformerBlock

GPT_CONFIG_124M = {
    "vocab_size": 50257,
    "context_length": 1024,
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 12,
    "drop_rate": 0.1,
    "qkv_bias": False,
}

block = TransformerBlock(GPT_CONFIG_124M)
print(block)

TransformerBlock(
  (att): MultiHeadAttention(
    (W_query): Linear(in_features=768, out_features=768, bias=False)
    (W_key): Linear(in_features=768, out_features=768, bias=False)
    (W_value): Linear(in_features=768, out_features=768, bias=False)
    (out_proj): Linear(in_features=768, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (ff): FeedForward(
    (layers): Sequential(
      (0): Linear(in_features=768, out_features=3072, bias=True)
      (1): GELU()
      (2): Linear(in_features=3072, out_features=768, bias=True)
    )
  )
  (norm1): LayerNorm()
  (norm2): LayerNorm()
  (drop_shortcut): Dropout(p=0.1, inplace=False)
)


In [4]:
total_params = sum(p.numel() for p in block.ff.parameters())
print(f"피드 포워드 모듈의 총 파라미터 수: {total_params:,}")

피드 포워드 모듈의 총 파라미터 수: 4,722,432


In [5]:
total_params = sum(p.numel() for p in block.att.parameters())
print(f"어텐션 모듈의 총 파라미터 수: {total_params:,}")

어텐션 모듈의 총 파라미터 수: 2,360,064


# 연습 문제 4.2: 더 큰 HPT 모델 초기화하기

In [7]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,
    "context_length": 1024,
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 12,
    "drop_rate": 0.1,
    "qkv_bias": False,
}

def get_config(base_config, model_name="gpt2-small"):
  GPT_CONFIG = base_config.copy()

  if model_name == "gpt2-small":
    GPT_CONFIG["emb_dim"] = 768
    GPT_CONFIG["n_layers"] = 12
    GPT_CONFIG["n_heads"] = 12
  elif model_name == "gpt2-medium":
    GPT_CONFIG["emb_dim"] = 1024
    GPT_CONFIG["n_layers"] = 24
    GPT_CONFIG["n_heads"] = 16
  elif model_name == "gpt2-large":
    GPT_CONFIG["emb_dim"] = 1280
    GPT_CONFIG["n_layers"] = 36
    GPT_CONFIG["n_heads"] = 20
  elif model_name == "gpt2-xl":
    GPT_CONFIG["emb_dim"] = 1600
    GPT_CONFIG["n_layers"] = 48
    GPT_CONFIG["n_heads"] = 25
  else:
    raise ValueError(f"잘못된 모델 이름입니다 {model_name}")

  return GPT_CONFIG

def calculate_size(model):

  total_params = sum(p.numel() for p in model.parameters())
  print(f"전체 파라미터 수: {total_params:,}")

  total_params_gpt2 = total_params - sum(p.numel() for p in model.out_head.parameters())
  print(f"가중치 묶기를 고려한 훈련 가능한 파라미터 수: {total_params_gpt2:,}")

  # 총 크기를 바이트 단위로 계산 (float32, 매개변수당 4바이트 가정)
  total_size_bytes = total_params * 4

  # 메가바이트로 변환
  total_size_mb = total_size_bytes / (1024 * 1024)

  print(f"모델의 총 크기: {total_size_mb:.2f} MB")

In [9]:
from gpt import GPTModel

for model_abbrev in ("small", "medium", "large", "xl"):
  model_name = f"gpt2-{model_abbrev}"
  CONFIG = get_config(GPT_CONFIG_124M, model_name=model_name)
  model = GPTModel(CONFIG)
  print(f"\n\n{model_name}:") # 모델 이름 출력
  calculate_size(model)



gpt2-small:
전체 파라미터 수: 163,009,536
가중치 묶기를 고려한 훈련 가능한 파라미터 수: 124,412,160
모델의 총 크기: 621.83 MB


gpt2-medium:
전체 파라미터 수: 406,212,608
가중치 묶기를 고려한 훈련 가능한 파라미터 수: 354,749,440
모델의 총 크기: 1549.58 MB


gpt2-large:
전체 파라미터 수: 838,220,800
가중치 묶기를 고려한 훈련 가능한 파라미터 수: 773,891,840
모델의 총 크기: 3197.56 MB


gpt2-xl:
전체 파라미터 수: 1,637,792,000
가중치 묶기를 고려한 훈련 가능한 파라미터 수: 1,557,380,800
모델의 총 크기: 6247.68 MB


# 연습문제 4.3: 별도의 드롭아웃 비율 사용하기

In [10]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,
    "context_length": 1024,
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 12,
    "drop_rate_emb": 0.1, # 임베딩 층에 대한 드롭아웃
    "drop_rate_attn": 0.1, # 멀티 헤드 어텐션에 대한 드롭아웃
    "drop_rate_shortcut": 0.1, # 숏컷 연결에 대한 드롭아웃
    "qkv_bias": False,
}

In [12]:
import torch.nn as nn
from gpt import MultiHeadAttention, LayerNorm, FeedForward

class TransformerBlock(nn.Module):
  def __init__(self, cfg):
    super().__init__()
    self.att = MultiHeadAttention(
        d_in=cfg["emb_dim"],
        d_out=cfg["emb_dim"],
        context_length=cfg["context_length"],
        num_heads=cfg["n_heads"],
        dropout=cfg["drop_rate_attn"], # 추가: 멀티 헤드 어텐션에 대한 드롭아웃
        qkv_bias=cfg["qkv_bias"],
    )
    self.ff = FeedForward(cfg)
    self.norm1 = LayerNorm(cfg["emb_dim"])
    self.norm2 = LayerNorm(cfg["emb_dim"])
    self.drop_shortcut = nn.Dropout(cfg["drop_rate_shortcut"])

  def forward(self, x):
    # 어텐션 블록을 위한 숏컷 연결
    shortcut = x
    x = self.norm1(x)
    x = self.att(x) # 크기 [batch_size, num_tokens, emb_size]
    x = self.drop_shortcut(x)
    x = x + shortcut # 원래 입력을 다시 더한다.

    # 피드포워드 블록을 위한 숏컷 연결
    shortcut = x
    x = self.norm2(x)
    x = self.ff(x)
    x = self.drop_shortcut(x)
    x = x + shortcut # 원래 입력을 다시 더한다.

    return x

class GPTModel(nn.Module):
  def __init__(self, cfg):
    super().__init__()
    self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
    self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
    self.drop_emb = nn.Dropout(cfg["drop_rate_emb"]) # 추가: 임베딩 층에 대한 드롭아웃

    self.trf_blocks = nn.Sequential(
        *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])]
    )

    self.final_norm = LayerNorm(cfg["emb_dim"])
    self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

  def forward(self, in_idx):
    batch_size, seq_len = in_idx.shape
    tok_embeds = self.tok_emb(in_idx)
    pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
    x = tok_embeds + pos_embeds # 크기 [batch_size, num_tokens, emb_size]
    x = self.drop_emb(x)
    x = self.trf_blocks(x)
    x = self.final_norm(x)
    logits = self.out_head(x)
    return logits

In [13]:
import torch

torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)